In [166]:
import sys
import os
import requests
import base64
import time
import json
import uuid
import random

In [167]:
IGZ_CONTAINER='bigdata'
INPUT_STREAM = 'pipeline_streams/input_stream'
V3IO_WEBAPI = 'v3io-webapi:8081'

In [168]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "http://%s/%s/%s/"% (V3IO_WEBAPI,IGZ_CONTAINER,INPUT_STREAM)
headers = {
            "Content-Type": "application/json",
            "X-v3io-function": "PutRecords",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
          }

In [169]:
def stream_frame_write(payload):
    bef = time.time()
    r = requests.post(url, headers=headers,json=payload, verify=False)   
    time_diff = time.time()-bef
    #print("Post time %s. Response %s"% (time_diff, r.text))
    return r.text

In [179]:
def send_message():
    Records=[]
    PartitionKey = str(uuid.uuid4())
   
    message={"PartitionKey" : PartitionKey,
            "pdf_path" : '/file/is/here',
            "s3_source" : 'picked://up/here'}
    messageb64 = base64.b64encode(json.dumps(message).encode('utf-8'))

    Records.append({
            "Data":  messageb64.decode('utf-8'),
            "PartitionKey": PartitionKey,

    })


    payload = {"Records": Records}
    r = stream_frame_write(payload)
    #print(PartitionKey)

In [180]:
msg_cnt = 0
while msg_cnt < 10000:
    send_message()
    msg_cnt += 1